In [11]:
import pandas as pd
import json
import datetime
import glob
import numpy as np

In [12]:
Store_Df=pd.read_csv("/home/jian/BiglotsCode/outputs/combined_sales_long_2018-07-28.csv",dtype=str)
Store_Df['week_end_date']=Store_Df['week_end_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
Store_Df=Store_Df[(Store_Df['week_end_date']>=datetime.date(2018,2,1))&(Store_Df['week_end_date']<=datetime.date(2018,5,6))]
Store_Df['sales']=Store_Df['sales'].astype(float)
Store_Df=Store_Df[Store_Df['sales']>0]


In [13]:
store_list_Q1=Store_Df['location_id'].unique().tolist()

In [14]:
store_list_1=pd.read_table("/home/jian/BiglotsCode/OtherInput/MediaStormStoreList_Nov15.txt",dtype=str,sep="|")
store_list_2=pd.read_table("/home/jian/BiglotsCode/OtherInput/MediaStormStores_20180703.txt",dtype=str,sep="|")
store_list_1['store_zip']=store_list_1['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))
store_list_2['store_zip']=store_list_2['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))

store_list_1=store_list_1[['location_id','store_zip','longitude_meas','latitude_meas']]
store_list_2=store_list_2[['location_id','store_zip','longitude_meas','latitude_meas']]
store_list_1=store_list_1[~store_list_1['location_id'].isin(store_list_2['location_id'])]
store_list=store_list_2.append(store_list_1)
store_list=store_list[store_list['location_id']!="6990"]
store_list=store_list[store_list['location_id']!="145"]


store_list['latitude_meas']=store_list['latitude_meas'].astype(float)
store_list['longitude_meas']=store_list['longitude_meas'].astype(float)

In [15]:
store_list=store_list[store_list['location_id'].isin(store_list_Q1)]

In [16]:
inclued_zips=store_list['store_zip'].unique().tolist()

In [17]:
weather_json_files=glob.glob("/home/jian/Projects/Big_Lots/Weather/Json_data/daily/"+"*.json")
weather_date=[datetime.datetime.strptime(x.split("/")[len(x.split("/"))-1][0:10],"%Y-%m-%d").date() for x in weather_json_files]
df_weather_files=pd.DataFrame({"path":weather_json_files,"date":weather_date},index=range(len(weather_json_files)))
df_weather_files['weekday']=df_weather_files['date'].apply(lambda x: x.weekday())
df_weather_files_weekends=df_weather_files[df_weather_files['weekday'].isin([5,6])].reset_index()
del df_weather_files_weekends['index']
df_weather_files_weekends['Month']=df_weather_files_weekends['date'].apply(lambda x: x.month)
df_weather_files_weekends=df_weather_files_weekends[(df_weather_files_weekends['date']>=datetime.date(2018,2,1)) & (df_weather_files_weekends['date']<datetime.date(2018,5,1))]

In [18]:
group_weight=pd.read_excel("/home/jian/Projects/Big_Lots/Weather/Q1_Weather_Counts/Q1_inclusion_store_all_weather_type_ranked.xlsx",sheetname="all_weather_group_list")
group_weight['Severity']=group_weight['Severity'].astype(int)
group_weight=group_weight[['all_type_group','Severity']]
group_weight_dict=group_weight[['all_type_group', 'Severity']].set_index('all_type_group').T.to_dict()

In [19]:
df_weather_files_weekends.head(2)

,date,path,weekday,Month
1,2018-02-18,/home/jian/Projects/Big_Lots/Weather/Json_data...,6,2
3,2018-04-07,/home/jian/Projects/Big_Lots/Weather/Json_data...,5,4


In [20]:
def choose_day_per_month(month):
    df=df_weather_files_weekends[df_weather_files_weekends['Month']==month]
    file_list=df['path'].unique().tolist()
    output_1_month=pd.DataFrame()
    for file in file_list:
        output_1_Day=pd.DataFrame()
        DATE=df[df['path']==file]['date'].unique()[0]
        response=json.load(open(file,"r"))
        for zip_cd in inclued_zips:
            if zip_cd in list(response.keys()):
                weather_list=response[zip_cd]['weather']
                dominated_weather_severity=0
                dominated_weather=np.nan
                for j in range(len(weather_list)):
                    weather_j=weather_list[j]['main']
                    weather_j_severity=group_weight_dict[weather_j]['Severity']
                    if weather_j_severity>dominated_weather_severity:
                        dominated_weather=weather_j
                        dominated_weather_severity=weather_j_severity
                        
                df_app=pd.DataFrame({"Date":datetime.datetime.fromtimestamp(response[list(response.keys())[0]]['dt']).date(),"zip_cd":zip_cd,"Weather":dominated_weather,'Severity':dominated_weather_severity},index=[0])
                output_1_Day=output_1_Day.append(df_app)
                output_1_Day['file_date']=DATE
        output_1_month=output_1_month.append(output_1_Day)
    return output_1_month
    
                

In [21]:
Feb_Severity=choose_day_per_month(2)
Feb_Severity_Avg=Feb_Severity.groupby(['Date'])['Severity'].mean().to_frame().reset_index()

In [22]:
Mar_Severity=choose_day_per_month(3)
Mar_Severity_Avg=Mar_Severity.groupby(['Date'])['Severity'].mean().to_frame().reset_index()

In [23]:
Apr_Severity=choose_day_per_month(4)
Apr_Severity_Avg=Apr_Severity.groupby(['Date'])['Severity'].mean().to_frame().reset_index()

In [24]:
zip_DMA=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,skiprows=1)
zip_DMA=zip_DMA.iloc[:,[0,2]]
zip_DMA.columns=['zip_cd','DMA']
zip_DMA=zip_DMA.drop_duplicates()

In [25]:
Feb_Severity_DMA_Duplicated=pd.merge(Feb_Severity,zip_DMA,on="zip_cd",how="left")
Mar_Severity_DMA_Duplicated=pd.merge(Mar_Severity,zip_DMA,on="zip_cd",how="left")
Apr_Severity_DMA_Duplicated=pd.merge(Apr_Severity,zip_DMA,on="zip_cd",how="left")

In [26]:
func = {'Severity':['mean'], 'zip_cd':['count']}

Feb_Severity_DMA_Day=Feb_Severity_DMA_Duplicated.groupby(['Date','DMA'])['Severity','zip_cd'].agg(func).reset_index()
Feb_Severity_DMA_Day.columns=[Feb_Severity_DMA_Day.columns.get_level_values(0).tolist()[i]+"_"+Feb_Severity_DMA_Day.columns.get_level_values(1).tolist()[i] for i in range(Feb_Severity_DMA_Day.shape[1])]

Mar_Severity_DMA_Day=Mar_Severity_DMA_Duplicated.groupby(['Date','DMA'])['Severity','zip_cd'].agg(func).reset_index()
Mar_Severity_DMA_Day.columns=[Mar_Severity_DMA_Day.columns.get_level_values(0).tolist()[i]+"_"+Mar_Severity_DMA_Day.columns.get_level_values(1).tolist()[i] for i in range(Mar_Severity_DMA_Day.shape[1])]

Apr_Severity_DMA_Day=Apr_Severity_DMA_Duplicated.groupby(['Date','DMA'])['Severity','zip_cd'].agg(func).reset_index()
Apr_Severity_DMA_Day.columns=[Apr_Severity_DMA_Day.columns.get_level_values(0).tolist()[i]+"_"+Apr_Severity_DMA_Day.columns.get_level_values(1).tolist()[i] for i in range(Apr_Severity_DMA_Day.shape[1])]


In [27]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Weather/Internal_Search_Phrase/Averaged_Weekend_Weather_Severity_by_Month_JL_20180801.xlsx",engine='xlsxwriter')
Feb_Severity_Avg.to_excel(writer,"Feb_Severity_Avg",index=False)
Mar_Severity_Avg.to_excel(writer,"Mar_Severity_Avg",index=False)
Apr_Severity_Avg.to_excel(writer,"Apr_Severity_Avg",index=False)

Feb_Severity.to_excel(writer,"Feb_Severity",index=False)
Mar_Severity.to_excel(writer,"Mar_Severity",index=False)
Apr_Severity.to_excel(writer,"Apr_Severity",index=False)

Feb_Severity_DMA_Day.to_excel(writer,"Feb_Severity_DMA_Day",index=False)
Mar_Severity_DMA_Day.to_excel(writer,"Mar_Severity_DMA_Day",index=False)
Apr_Severity_DMA_Day.to_excel(writer,"Apr_Severity_DMA_Day",index=False)

writer.save()







In [190]:
# check
file="/home/jian/Projects/Big_Lots/Weather/Json_data/daily/2018-02-12.json"
response=json.load(open(file,"r"))
response['82001']

{'base': 'stations',
 'clouds': {'all': 75},
 'cod': 200,
 'coord': {'lat': 41.14, 'lon': -104.8},
 'dt': 1518458280,
 'id': 0,
 'main': {'humidity': 92,
  'pressure': 1015,
  'temp': 260.15,
  'temp_max': 260.15,
  'temp_min': 260.15},
 'name': 'Cheyenne',
 'sys': {'country': 'US',
  'id': 3074,
  'message': 0.0043,
  'sunrise': 1518443793,
  'sunset': 1518481848,
  'type': 1},
 'visibility': 4426,
 'weather': [{'description': 'mist', 'icon': '50d', 'id': 701, 'main': 'Mist'},
  {'description': 'light snow', 'icon': '13d', 'id': 600, 'main': 'Snow'}],
 'wind': {'deg': 107.003, 'speed': 2.21}}

In [118]:
df_weather_files_weekends['path'][2]

'/home/jian/Projects/Big_Lots/Weather/Json_data/daily/2018-02-18.json'